In [1]:
import data
import models
import cache
import seaborn as sns
import numpy as np
import pandas as pd
import stanity

/home/jacquelineburos/miniconda3/envs/immuneinf/lib/python3.5/site-packages/Cython/Distutils/old_build_ext.py:30: UserWarning: Cython.Distutils.old_build_ext does not properly handle dependencies and is deprecated.
  "Cython.Distutils.old_build_ext does not properly handle dependencies "
INFO:stancache.seed:Setting seed to 1245502385
INFO:root:Setting CACHE_DIR = /mnt/modelcache/immune-infiltrate-explorations
INFO:stancache.seed:Setting seed to 1245502385


In [2]:
sns.set(context='talk')

In [4]:
by = 'SubSet'
sample_n = 500

## get data, as we did in earlier examples

This will help in case we want to compare estimates for particular genes or samples

In [5]:
sample_df = cache.cached(models.prep_sample_df, sample_n=sample_n)

INFO:stancache.stancache:prep_sample_df: cache_filename set to prep_sample_df.cached.sample_n_500.pkl
INFO:stancache.stancache:prep_sample_df: Loading result from cache


In [6]:
stan_data = models.prep_stan_data(sample_df, by=by)

## get models from cache

In [7]:
model4 = models.get_model_file(model_name='model4')
fit4 = models.cached_stan_fit(file=model4, data=stan_data, model_name='model4', cache_only=True)

INFO:stancache.stancache:Step 1: Get compiled model code, possibly from cache
INFO:stancache.stancache:StanModel: cache_filename set to model4.cython_0_25_1.model_code_1623578005189906009.pystan_2_12_0_0.stanmodel.pkl
INFO:stancache.stancache:StanModel: Loading result from cache
INFO:stancache.stancache:Step 2: Get posterior draws from model, possibly from cache
INFO:stancache.stancache:sampling: cache_filename set to model4.cython_0_25_1.model_code_1623578005189906009.pystan_2_12_0_0.stanfit.chains_4.data_28701051159.iter_2000.seed_1245502385.pkl
INFO:stancache.stancache:sampling: Loading result from cache


In [8]:
fit4_prefix = models.cached_stan_file(file=model4, data=stan_data, model_name='model4', prefix_only=True)
fit4_prefix

'model4.cython_0_25_1.model_code_1623578005189906009.pystan_2_12_0_0.stanfit.data_28701051159'

In [9]:
model5 = models.get_model_file(model_name='model5')
fit5 = models.cached_stan_fit(file=model5, data=stan_data, model_name='model5', cache_only=True)

INFO:stancache.stancache:Step 1: Get compiled model code, possibly from cache
INFO:stancache.stancache:StanModel: cache_filename set to model5.cython_0_25_1.model_code_12673779526111968781.pystan_2_12_0_0.stanmodel.pkl
INFO:stancache.stancache:StanModel: Loading result from cache
INFO:stancache.stancache:Step 2: Get posterior draws from model, possibly from cache
INFO:stancache.stancache:sampling: cache_filename set to model5.cython_0_25_1.model_code_12673779526111968781.pystan_2_12_0_0.stanfit.chains_4.data_28701051159.iter_2000.seed_1245502385.pkl


ValueError: sampling: Cachefile does not exist and cache_only == True. Exiting with failure.

In [11]:
fit5_prefix = models.cached_stan_file(file=model5, data=stan_data, model_name='model5', prefix_only=True)
fit5_prefix

'model5.cython_0_25_1.model_code_12673779526111968781.pystan_2_12_0_0.stanfit.data_28701051159'

## compute & compare psis-loo for each model

In [10]:
loo4 = cache.cached(stanity.psisloo,
                    log_likelihood=fit4.extract('log_lik')['log_lik'],
                    cache_filename='{}.loo.pkl'.format(fit4_prefix))
loo4.print_summary()

INFO:stancache.stancache:psisloo: cache_filename set to model4.cython_0_25_1.model_code_1623578005189906009.pystan_2_12_0_0.stanfit.data_28701051159.loo.pkl
INFO:stancache.stancache:psisloo: Starting execution
/home/jacquelineburos/miniconda3/envs/immuneinf/lib/python3.5/site-packages/stanity/psis.py:228: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif sort == 'in-place':
/home/jacquelineburos/miniconda3/envs/immuneinf/lib/python3.5/site-packages/stanity/psis.py:246: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  bs /= 3 * x[sort[np.floor(n/4 + 0.5) - 1]]
/home/jacquelineburos/miniconda3/envs/immuneinf/lib/python3.5/site-packages/stanity/psis.py:262: RuntimeWarning: overflow encountered in exp
  np.exp(temp, out=temp)
INFO:stancache.stancache:psisloo: Execution completed (0:02:13.969050 elapsed)
INFO:stancache.stancache:psisloo: Savin

greater than 0.5    0.042952
greater than 1      0.001937
dtype: float64

In [ ]:
loo5 = cache.cached(stanity.psisloo,
                    log_likelihood=fit5.extract('log_lik')['log_lik'],
                    cache_filename='{}.loo.pkl'.format(fit5_prefix))
loo5.print_summary()

In [ ]:
stanity.loo_compare(loo4, loo5)